# Import Required Libraries
Import necessary libraries including nltk, threading, json, pickle, numpy, tensorflow, and custom modules.

In [ ]:
import nltk
import threading
import json
import pickle
import numpy as np
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
import dashboard
from dashboard import run_dashboard
from analytics import AnalyticsTracker

nltk.download('punkt')
nltk.download('wordnet')

# Download NLTK Data
Download required NLTK data such as 'punkt' and 'wordnet'.

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')

# Initialize Lemmatizer
Initialize the WordNetLemmatizer.

In [ ]:
lemmatizer = WordNetLemmatizer()

# Start Dashboard Thread
Start the dashboard thread using threading.

In [ ]:
dashboard_thread = threading.Thread(target=run_dashboard, daemon=True)
dashboard_thread.start()

# Define get_response Function
Define the get_response function to handle user input and log interactions.

In [ ]:
tracker = AnalyticsTracker()

def get_response(user_input):
    intent = "detected_intent"  
    response = "generated_response"  
    
    tracker.log_interaction(
        user_id="current_user",  
        query=user_input,
        intent=intent,
        response=response
    )
    
    return response

# Load Intents JSON
Load the intents JSON file containing chatbot intents.

In [ ]:
intents = json.load(open(r'E:\chatbot\data\intents.json'))

# Preprocess Data
Tokenize and lemmatize words, and prepare documents and classes for training.

In [ ]:
words = []
classes = []
documents = []
ignoreLetters = ["?", "!", ".", ","]

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent["tag"]))
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

# Save Preprocessed Data
Save the preprocessed words and classes using pickle.

In [ ]:
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

# Prepare Training Data
Prepare the training data by creating bags of words and corresponding output classes.

In [ ]:
training = []
outputEmpty = [0] * len(classes)

for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)
        
    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)
    
random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]

# Train the Model
Define and train the neural network model using TensorFlow.

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation="relu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(classes), activation="softmax"))

sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

hist = model.fit(np.array(trainX), np.array(trainY), epochs=400, batch_size=5, verbose=1)
model.save("chatbot_model.h5", hist)

# Test Interaction
Test the chatbot interaction with a sample input.

In [ ]:
test_input = "Hello chatbot!"
print("Chatbot Response:", get_response(test_input))
print("Interaction logged in analytics!")

# Log Interaction
Define a function to log interactions to a JSON file.

In [ ]:
import datetime

def log_interaction(user_query, bot_response):
    log_data = {
        "timestamp": str(datetime.datetime.now()),
        "query": user_query,
        "response": bot_response
    }

    with open("chat_logs.json", "a") as logfile:
        logfile.write(json.dumps(log_data) + "\n")

    print(f"Interaction Logged: {log_data}")